In [1]:
import networkx as nx
import numpy as np
import Visualizer

In [2]:
G = nx.gnm_random_graph(2000, 10000, seed=42)

In [3]:
def simulate_sirs(G,beta,gamma,xi,f,lst_initial_immune,lst_initial_infected):
    attDict = {}
    for node in lst_initial_immune:
        attDict[node] = [0]
    for node in lst_initial_infected:
        attDict[node] = [1]
    t = 0
    D = []
    D.append([len(lst_initial_immune), len(lst_initial_infected), 0])
    I = 645
    #Uncomment when creating newGraph: while t < f and I < 644:
    #Comment when simulating SIRS: while t < f: #and I < 644:
    while t < f: #and I < 644:
        S = 0
        I = 0
        R = 0
        for node in list(attDict.keys()):
            if attDict[node][t] == 0:
                if np.random.random() < beta:
                    attDict[node].append(1)
                    I += 1
                else:
                    attDict[node].append(0)
                    S += 1
            if attDict[node][t] == 1:
                if np.random.random() < gamma:
                    attDict[node].append(2)
                    R += 1
                else:
                    attDict[node].append(1)
                    I += 1
            if attDict[node][t] == 2:
                if np.random.random() < xi:
                    attDict[node].append(0)
                    S += 1
                else:
                    attDict[node].append(2)
                    R += 1
        D.append([S,I,R])
        t+=1
    nx.set_node_attributes(G,attDict,"state")
    infected = []
    for node in G.nodes():
        if G.nodes[node]["state"][t-1] == 1:
            infected.append(node)
    if len(infected) < 644:
        newGraph = G.subgraph(infected)
    else:
        newGraph = G.subgraph(np.random.choice(infected, 644, replace=False))
    return (D, newGraph)

In [4]:
sim = simulate_sirs(G, 0.25, 0.4, 0.1, 15, list(G.nodes)[:1000], list(G.nodes)[1000:])

In [5]:
D = sim[0]
ngraph = sim[1]
(len(ngraph.nodes()), len(ngraph.edges()))

(277, 210)

In [23]:
char_path_length = 'not connected'
pos_max = np.argmax([len(listElem) for listElem in sorted(nx.connected_components(ngraph))])
char_path_length_component = [nx.average_shortest_path_length(ngraph.subgraph(g)) for g in sorted(nx.connected_components(ngraph))][pos_max]

density = nx.density(ngraph)
degree = sum(dict(ngraph.degree()).values())/len(ngraph)
transitivity = nx.transitivity(ngraph)
clustering_coef = nx.average_clustering(ngraph)

n_connected_comp = nx.number_connected_components(ngraph)
size_connected_comp = sum([len(listElem) for listElem in sorted(nx.connected_components(ngraph))])/len(sorted(nx.connected_components(ngraph)))

print(f'original characteristic path length {char_path_length}, \noriginal characteristic path length of biggest component {char_path_length_component},\noriginal network density {density},\noriginal network average degree {degree},\noriginal transitivity {transitivity}, \noriginal average clustering coefficient {clustering_coef}, \noriginal number of connected components {n_connected_comp}, \noriginal average size of a connected component {size_connected_comp}')

original characteristic path length not connected, 
original characteristic path length of biggest component 5.414217019053407,
original network density 0.0051341247838644554,
original network average degree 3.301242236024845,
original transitivity 0.005229517722254503, 
original average clustering coefficient 0.004611554766834272, 
original number of connected components 28, 
original average size of a connected component 23.0


In [8]:
sim_ngraph = simulate_sirs(ngraph, 0.25, 0.05, 0.0001, 35, list(ngraph.nodes)[:640], list(ngraph.nodes)[640:])

In [9]:
Visualizer.showSIRS(ngraph, "random_graph_simulation.html", 0.25, 0.05, 0.0001, 35, sim_ngraph[0])

Nodes positions computed.
Edges trace computed.
Nodes trace computed.
Frames computed.
Figure generated.
Visualization rendering done
File saved.


In [10]:
Visualizer.showData(sim_ngraph[0], "random_model_data.html", 0.25, 0.05, 0.0001)

In [12]:
import pickle
with open('results_random.pkl', 'wb') as outfile:
    pickle.dump(sim_ngraph, outfile)